# **ТГ-Бот “Д’Артаньян”**

На определённых этапах изучения языка количество выучиваемых слов становится всё больше и больше. В какой-то момент можно даже прийти к тому, что это делать всё-таки лень. Наш бот предлагает не учить непосредственно перевод каких-то слов, а просто отмечать какую характеристику несёт это слово: положительную или отрицательную. Также подобный подход может пригодится в условиях подготовки к экзамену, когда надо понимать не столько сам перевод слова, сколько эмоциональную окраску (за/против, да/нет, положительно/отрицательно).

В данной тетрадке представлен первый этап нашей работы: выбор, обучение и сохранение результата работы модели.

Для начала, скачаем датасет. Мы выбрали [датасет](https://huggingface.co/datasets/tblard/allocine) с [Hugging Face](https://huggingface.co/). Основные преимущества данного датасета:

1.   Выборка скачана с французского сайта. Данные не являются переводом с русского/английского языков, а представляют собой аутентичный материал.
2.   Датасет имеет равное количество данных как для положительных отзывов, так и для отрицательных. Это поможет модели равномерно обучиться на обоих вариантах тональности.
3.   Собранные в датасете данные представляют собой живой язык, язык повседневного общения и слегка "отходит" от формальных конструкций, предлагаемых в учебниках. Это поможет нашем пользователям погрузиться в мир "настоящих французов".

*Théophile Blard, French sentiment analysis with BERT, (2020), GitHub repository, https://github.com/TheophileBlard/french-sentiment-analysis-with-bert*

## Датасет

In [1]:
!wget https://github.com/TheophileBlard/french-sentiment-analysis-with-bert/blob/master/allocine_dataset/data.tar.bz2

--2024-06-25 19:38:54--  https://github.com/TheophileBlard/french-sentiment-analysis-with-bert/blob/master/allocine_dataset/data.tar.bz2
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data.tar.bz2’

data.tar.bz2            [ <=>                ] 268.13K  --.-KB/s    in 0.1s    

2024-06-25 19:38:55 (2.69 MB/s) - ‘data.tar.bz2’ saved [274567]



In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

в

In [3]:
from datasets import load_dataset
import pandas as pd

In [4]:
dataset = load_dataset('allocine')
df = pd.DataFrame(dataset['train'])
df.to_csv('train.csv', index=False)

df_val = pd.DataFrame(dataset['validation'])
df_val.to_csv('validation.csv', index=False)

df_test = pd.DataFrame(dataset['test'])
df_test.to_csv('test.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/160000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20000 [00:00<?, ? examples/s]

## Обработка текста

Перейдем к предобработке текста. Для начала выполним все нужные импорты. Сам процесс предобработки заключатеся в том, чтобы привести текст к нижнему регистру и убрать все лишние символы. Мы не удаляем стоп-слова, т.к. это может привести к выбору неверной тональности.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import re
import string

In [6]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation.replace("'", "")), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df['review'] = df['review'].apply(preprocess_text)
df_val['review'] = df_val['review'].apply(preprocess_text)
df_test['review'] = df_test['review'].apply(preprocess_text)

## Выбор модели

Для сравнения мы выбрали 3 модели: LogisticRegression(), MultinomialNB(), RandomForestClassifier().

Вот краткое описание преимуществ каждой из этих моделей при выполнении задачи определения тональности:

1. Logistic Regression:
   - Логистическая регрессия является простым и эффективным алгоритмом для задач классификации, включая задачу определения тональности.
   - Хорошо работает с линейно разделимыми данными.
   - Дает вероятностную интерпретацию результатов.

2. Multinomial Naive Bayes:
   - Мультиномиальный наивный Байесовский классификатор хорошо работает с текстовыми данными, что делает его хорошим выбором для анализа тональности текста.
   - Эффективен при работе с большими корпусами текста.
   - Хорошо справляется с множеством признаков.

3. Random Forest Classifier:
   - Случайный лес является мощным алгоритмом машинного обучения, который хорошо подходит для задач классификации, включая определение тональности.
   - Способен обрабатывать большое количество признаков и автоматически находить наиболее важные.
   - Устойчив к переобучению и хорошо работает на больших объемах данных.

In [ ]:
models = [
    LogisticRegression(max_iter=1000),
    MultinomialNB(),
    RandomForestClassifier()
]

for model in models:
    print(model)
    pipeline = make_pipeline(TfidfVectorizer(), model)
    pipeline.fit(df['review'], df['label'])
    predictions = pipeline.predict(df_test['review'])
    print(classification_report(df_test['label'], predictions))
    print('ROC_AUC score: ', roc_auc_score(df_test['label'], predictions))
    print('\n')

LogisticRegression(max_iter=1000)
              precision    recall  f1-score   support

           0       0.93      0.92      0.93     10408
           1       0.92      0.92      0.92      9592

    accuracy                           0.92     20000
   macro avg       0.92      0.92      0.92     20000
weighted avg       0.92      0.92      0.92     20000

ROC_AUC score:  0.924557014588765


MultinomialNB()
              precision    recall  f1-score   support

           0       0.91      0.90      0.91     10408
           1       0.89      0.91      0.90      9592

    accuracy                           0.90     20000
   macro avg       0.90      0.90      0.90     20000
weighted avg       0.90      0.90      0.90     20000

ROC_AUC score:  0.9026684580219618


RandomForestClassifier()
              precision    recall  f1-score   support

           0       0.88      0.90      0.89     10408
           1       0.89      0.87      0.88      9592

    accuracy                      

Мы выяснили, что логистическая регрессия показала себя лучше всех. Теперь нужно подобрать верные параметры.

## Подбор параметров

Зафиксируем наш изначальный результат логистической регрессии до подобранных параметров.

In [67]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train = vectorizer.fit_transform(df['review'])
X_test = vectorizer.transform(df_test['review'])
X_val = vectorizer.transform(df_val['review'])

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, df['label'])

y_test_pred = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(df_test['label'], y_test_pred)
print('ROC AUC score:', roc_auc)

ROC AUC score: 0.9826529634290426


Для подбора параметров воспользуемс методом GridSearchCV.

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['lbfgs', 'sag', 'saga'],
    'class_weight': [None, 'balanced'],
    'max_iter':[1000, 1500]
}


grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=3, verbose=2, scoring='roc_auc')
grid_search.fit(X_val, df_val['label'])


print('Best parameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=lbfgs; total time=  17.1s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=lbfgs; total time=  12.5s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=lbfgs; total time=  13.6s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=sag; total time=   2.5s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=sag; total time=   2.5s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=sag; total time=   3.5s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=saga; total time=   3.7s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=saga; total time=   3.6s
[CV] END C=0.1, class_weight=None, max_iter=1000, solver=saga; total time=   3.7s
[CV] END C=0.1, class_weight=None, max_iter=1500, solver=lbfgs; total time=  12.3s
[CV] END C=0.1, class_weight=None, max_iter=1500, solver=lbfgs; total time=  15.1s
[CV] END C=0.1, class_weight=None,

Подставим получившийся результат. Мы получим следующий ROC_AUC, который улучшился по сравнению с прошлым. Ура, победа!

In [68]:
model = LogisticRegression(C=10, class_weight='balanced',solver='saga', max_iter=1500)
model.fit(X_train, df['label'])

y_test_pred = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(df_test['label'], y_test_pred)
print('ROC AUC score:', roc_auc)

ROC AUC score: 0.9867574959981383


## Обучение финальной модели и сохранение



Окончательная версия модели была обучена на всех данных (train, test, validation), включая суммарно 200 000 записей, по следующим причинам:

1. Увеличение разнообразия данных: Объединение всех данных из тренировочного, тестового и валидационного наборов позволяет увеличить разнообразие данных, на которых модель обучается. Это может помочь модели лучше обобщать и обрабатывать различные типы запросов или задач.

2. Улучшение общей производительности: Обучение модели на большем количестве данных может привести к улучшению ее общей производительности. Больший объем данных позволяет модели выявлять более сложные зависимости и шаблоны в данных, что может привести к лучшим результатам при выполнении различных задач.

В целом, обучение окончательной версии модели на всех доступных данных является стратегией, которая может помочь улучшить производительность, обобщающую способность и стабильность модели при выполнении различных задач.

In [7]:
total_df = pd.concat([df, df_test, df_val], axis=0)

In [8]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
total_train = vectorizer.fit_transform(total_df['review'])

In [9]:
model = LogisticRegression(C=10, class_weight='balanced',solver='saga', max_iter=1500)
model.fit(total_train, total_df['label'])

LogisticRegression(C=10, class_weight='balanced', max_iter=1500, solver='saga')

Сохраняем модель в pickle

In [ ]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

Сохраняем векторизатор в pickle

In [ ]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

Сохраняем feature names

In [ ]:
feature_names = vectorizer.get_feature_names_out()
np.save('feature_names.npy', feature_names)

## Примеры работы модели

Посмотрим на самые важные слова для модели при определении положительных и отрицательных отызвов.

In [ ]:
feature_names = vectorizer.get_feature_names_out()
np.save('feature_names.npy', feature_names)

coefficients = model.coef_[0]
important_words = pd.DataFrame({'word': feature_names, 'coefficient': coefficients})
important_words = important_words.sort_values(by='coefficient', ascending=False)

print("Top positive words:")
print(important_words.head(10))

print("Top negative words:")
print(important_words.tail(10))

Top positive words:
                 word  coefficient
1011183     excellent    23.177288
1664413    magnifique    19.949082
2702316       superbe    17.176937
2015261       pas mal    14.975856
2859991      très bon    13.778212
263733       bon film    13.759377
1990762  parfaitement    12.910640
1240539        génial    12.813046
1737289     merveille    12.600889
1989548       parfait    12.295889
Top negative words:
              word  coefficient
2400067       rien   -14.912549
2398239   ridicule   -15.332302
824693    décevant   -15.403648
2124752        plv   -15.733717
1911656        nul   -15.784441
824074   déception   -17.640367
1386245    intérêt   -18.450239
1859128      navet   -18.645419
1716689    mauvais   -20.261444
932378    ennuyeux   -20.412900


Теперь рассмотрим как предсказывать тональность одного отзыва (моделируем ситуацию использования бота).

In [34]:
def predict_sentiment(model, vectorizer, text):
    preprocessed_text = preprocess_text(text)
    text_vector = vectorizer.transform([preprocessed_text])
    probability = model.predict_proba(text_vector)[:, 1]
    line = str(str(*probability * 100)[:5]) + '%'

    return line

In [35]:
predict_sentiment(model, vectorizer, "C'est un film mauvais imprégné de haine! Je refuse de regarder ça.")

'8.319%'

Вот слова, котоые показались модели самыми важными при определении тональности.

In [40]:
text_vector = vectorizer.transform([preprocess_text("C'est un film mauvais imprégné de haine! Je refuse de regarder ça.")])

contributions = text_vector.multiply(model.coef_[0])

feature_names = vectorizer.get_feature_names_out()

import pandas as pd
contributions_df = pd.DataFrame({'Feature': feature_names, 'Contribution': contributions.toarray()[0]})
contributions_df = contributions_df.sort_values(by='Contribution', ascending=False)

print(list(contributions_df.tail(2)['Feature']))

['regarder', 'mauvais']


In [19]:
print(list(contributions_df.head(2)['Feature']))

['beau film', 'beau']
